In [36]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
from datetime import datetime, timedelta
import psycopg2
from psycopg2 import sql

DB_HOST = 'localhost'
DB_NAME = 'tw'
DB_USER = 'mini'
DB_PASS = 'buddyrich134'
def get_db_connection():
    conn = psycopg2.connect(host=DB_HOST, dbname=DB_NAME, user=DB_USER, password=DB_PASS)
    return conn

conn = get_db_connection()
cursor = conn.cursor()

In [11]:
cursor.execute(sql.SQL("Select * from public.block_trade;"))
conn.commit()
res = cursor.fetchall()
final_df = pd.DataFrame(res)
final_df.columns = ['證券代號', '證券名稱', '交易類別', '成交價格', '成交股數', '成交金額', '日期']
rolling = 365
group_data = []
for group_name, group_df in tqdm(final_df.groupby('證券代號'), desc='loop by symbol'):
    if group_name == "總計":continue
    group_df.sort_values(by='日期', ascending=True, inplace=True)
    group_df['年均價'] = 0.0
    for last_date in group_df['日期']:
        start_date = last_date - timedelta(days=rolling)
        time_df = group_df[(group_df['日期'] >= start_date) & (group_df['日期'] <= last_date)]
        total_amount = time_df['成交金額'].sum()
        total_volume = time_df['成交股數'].sum()
        avg_price = total_amount/total_volume
        
        group_df.loc[group_df['日期'] == last_date, '年均價'] = avg_price
    group_data.append(group_df)
df = pd.concat(group_data)
df_drop = df.dropna(axis=1).reset_index(drop=True)
block_trade = df_drop.copy()
block_trade

loop by symbol: 100%|██████████| 743/743 [00:22<00:00, 32.74it/s]


,index,證券代號,證券名稱,交易類別,成交價格,成交股數,成交金額,日期,年均價
0,78760,0050,元大台灣50,配對交易,81.81,200000,16362000,2018-02-23,81.81000
1,78857,0050,元大台灣50,配對交易,81.85,200000,16370000,2018-03-06,81.83000
2,79047,0050,元大台灣50,配對交易,82.07,200000,16414000,2018-03-23,81.91000
3,75292,0050,元大台灣50,配對交易,76.22,200000,15244000,2019-01-21,79.63400
4,57653,0050,元大台灣50,配對交易,76.22,200000,15244000,2019-01-21,79.63400
...,...,...,...,...,...,...,...,...,...
81938,13007,9958,世紀鋼,配對交易,170.50,502000,85591000,2023-10-20,138.08285
81939,21860,9958,世紀鋼,配對交易,170.50,502000,85591000,2023-10-20,138.08285
81940,33570,9958,世紀鋼,配對交易,170.50,502000,85591000,2023-10-20,138.08285
81941,48009,9958,世紀鋼,配對交易,170.50,502000,85591000,2023-10-20,138.08285


In [ ]:
# add close price
pivoted_cl = pd.read_parquet("cl_2019.parquet")
block_trade['cl'] = None
def block_trade_cl(row):
    code, da = row['證券代號'], row['日期']
    print(code, da)
    return pivoted_cl[code][da]
block_trade['cl'] = block_trade.apply(block_trade_cl, axis=1)

block_trade_sortda = block_trade.sort_values(by='日期', ascending=True)
def calc_factor(row):
    avg, cl = row['年均價'], row['cl']
    try:
        diff = avg/cl
        # if diff > 0.3: print(avg, cl, row['日期'], row['證券代號'])
        return diff if diff > 1.3 else 0.0
    except ZeroDivisionError as e:
        return 0.0
concat_list = []
for enum, (i, j) in enumerate(block_trade_sortda.groupby('日期')):
    j.drop_duplicates(subset=['證券代號', '證券名稱'], inplace=True)
    j['factor'] = j.apply(calc_factor, axis=1)
    concat_list.append(j)
concated_pd = pd.concat(concat_list)
concated_pd

In [20]:
url_json = "https://www.twse.com.tw/rwd/zh/afterTrading/BWIBBU_d?date=20240809&selectType=ALL&response=json"
res = requests.get(url_json)
code = [i[0] for i in res.json()['data']]
cname = [i[1] for i in res.json()['data']]

In [35]:
import yfinance as yf
df = yf.download('1101.TW', interval='5m', start='2024-07-01', progress=False, )
df.index = df.index.tz_convert(None)
df.index = df.index + timedelta(hours=8)
df[:30]

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2024-07-01 09:00:00,33.599998,33.700001,33.549999,33.650002,33.650002,0
2024-07-01 09:05:00,33.650002,33.799999,33.599998,33.750000,33.750000,1034931
2024-07-01 09:10:00,33.799999,33.849998,33.700001,33.700001,33.700001,1089742
2024-07-01 09:15:00,33.750000,33.750000,33.549999,33.599998,33.599998,1359415
2024-07-01 09:20:00,33.599998,33.650002,33.549999,33.650002,33.650002,339391
2024-07-01 09:25:00,33.650002,33.650002,33.599998,33.650002,33.650002,317188
2024-07-01 09:30:00,33.650002,33.700001,33.599998,33.700001,33.700001,226334
2024-07-01 09:35:00,33.700001,33.750000,33.650002,33.700001,33.700001,375753
2024-07-01 09:40:00,33.700001,33.700001,33.650002,33.650002,33.650002,213284


In [50]:
df['本益比'][0]

'31.08'

In [62]:
df_row = []
for i in range(1, datetime.now().month+1):
    try:
        da = f"0{i}" if i < 10 else i
        date = f"202408{da}"
        url_json = f"https://www.twse.com.tw/rwd/zh/afterTrading/BWIBBU_d?date={date}&selectType=ALL&response=json"
        res = requests.get(url_json)
        data = res.json()['data']
        df  = pd.DataFrame(data)
        df.columns = res.json()['fields']
        df['本益比'] = df['本益比'].apply(lambda x: x.replace(",", ''))
        df = df.replace("-", 1.0)
        df['殖利率(%)'] = df['殖利率(%)'].astype(float)
        df['本益比'] = df['本益比'].astype(float)
        df['股價淨值比'] = df['股價淨值比'].astype(float)
        df['factor'] = df['殖利率(%)'] * 1/df['本益比'] * 1/df['股價淨值比']
        df.sort_values(by='factor',ascending=False, inplace=True)
        df['da'] = f"2024-08-{da}"
        df_row.append(df.iloc[0, :])
    except:
        pass

cursor.execute(sql.SQL("Select * from public.block_code3_deatil where strategy = 'pe_pb_yield' order by da desc limit 1;"))
conn.commit()
res = cursor.fetchone()
df_signal = pd.DataFrame(df_row)
df_signal = df_signal[['證券代號', 'da', '本益比', 'factor', 'factor']]
df_signal.columns = ['code', 'da', 'cl', 'strategy', 'signal']
df_signal['strategy'] = 'pe_pb_yield'
df_signal['signal'] = 1
if not res:
    self.
else:
    df_signal = df_signal[df_signal['da'] > res[0]]

[證券代號            1445
 證券名稱              大宇
 殖利率(%)         14.41
 股利年度             112
 本益比             2.28
 股價淨值比           0.85
 財報年/季          113/1
 factor      7.435501
 da        2024-08-01
 Name: 74, dtype: object,
 證券代號            1445
 證券名稱              大宇
 殖利率(%)         14.79
 股利年度             112
 本益比             2.22
 股價淨值比           0.82
 財報年/季          113/1
 factor      8.124588
 da        2024-08-02
 Name: 74, dtype: object,
 證券代號            1445
 證券名稱              大宇
 殖利率(%)         16.29
 股利年度             112
 本益比             2.02
 股價淨值比           0.75
 財報年/季          113/1
 factor     10.752475
 da        2024-08-05
 Name: 74, dtype: object,
 證券代號            1445
 證券名稱              大宇
 殖利率(%)         16.67
 股利年度             112
 本益比             1.97
 股價淨值比           0.73
 財報年/季          113/1
 factor     11.591683
 da        2024-08-06
 Name: 74, dtype: object,
 證券代號            1445
 證券名稱              大宇
 殖利率(%)         15.82
 股利年度             112
 本益比            

In [70]:
df_signal = pd.DataFrame(df_row)
df_signal = df_signal[['證券代號', 'da', '本益比', 'factor', 'factor']]
df_signal.columns = ['code', 'da', 'cl', 'strategy', 'signal']
df_signal['strategy'] = 'pe_pb_yield'
df_signal['signal'] = 1
df_signal

,code,da,cl,strategy,signal
74,1445,2024-08-01,2.28,pe_pb_yield,1
74,1445,2024-08-02,2.22,pe_pb_yield,1
74,1445,2024-08-05,2.02,pe_pb_yield,1
74,1445,2024-08-06,1.97,pe_pb_yield,1
74,1445,2024-08-07,2.10,pe_pb_yield,1
74,1445,2024-08-08,2.05,pe_pb_yield,1
413,2537,2024-08-09,1.00,pe_pb_yield,1
330,2409,2024-08-12,1.00,pe_pb_yield,1


In [67]:
cursor.execute(sql.SQL("Select * from public.block_code3_deatil where strategy = 'pe_pb_yield' order by da desc limit 1;"))
conn.commit()
res = cursor.fetchone()
df_signal = pd.DataFrame(df_row)
df_signal = df_signal[['證券代號', 'da', '本益比', 'factor', 'factor']]
df_signal.columns = ['code', 'da', 'cl', 'strategy', 'signal']
df_signal['strategy'] = 'pe_pb_yield'
df_signal['signal'] = 1
if not res:
    self.
else:
    df_signal = df_signal[df_signal['da'] > res[0]]

1


In [73]:

def _( code):
    list_concat = []
    df_final = []
    for year in range(2024, 2023, -1):
        limit_month = 9 if 2024 == 2024 else 12
        for _, month in enumerate(range(limit_month, 0, -1)):
            try:
                month = f"0{month}" if month < 10 else month
                da = f"{year}{month}01"
                url_json = f"https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date={da}&stockNo={code}"
                response = requests.get(url_json)
                list_concat.append(response.json()['data'])
            except:
                if month == 1:
                    return []
                else:
                    list_concat = [item for item in list_concat if item is not None]                                        
                    break
    try:
        df_final = pd.DataFrame(sum(list_concat, []))
        df_final = df_final.iloc[:, 0:-2]
        df_final.columns = ['da', 'vol_share', 'vol', 'op', 'hi', 'lo', 'cl']
        df_final.replace("--", None, inplace=True)
        df_final.replace("-", None, inplace=True)
        df_final.ffill(inplace=True)
        def convert_to_2024(da):
            year, month, day = da.split("/")
            return f"{int(year)+1911}-{month}-{day}"
        df_final['da'] = df_final['da'].apply(convert_to_2024)
        for i in range(1, len(df_final.columns)):
            df_final.iloc[:, i] = df_final.iloc[:, i].apply(lambda x: float(x.replace(",", "")))
        df_final['vol_share'] = df_final['vol_share'].astype(int) 
        df_final['vol'] = df_final['vol'].astype(int) 
        df_final['code'] = code
        return df_final 
    except Exception as e:
        print(e)
        return []

In [74]:
cursor.execute(sql.SQL("Select code from public.maincode where listed = 'TW';"))
conn.commit()
res = cursor.fetchall()
stock_list = [i[0] for i in res]
list_df = []
for code in tqdm(stock_list):
    df = _(code)
    list_df.append(df)
final_df = pd.DataFrame(list_df)

100%|██████████| 1119/1119 [01:04<00:00, 17.34it/s]


In [77]:
list_df

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
